#This code creates a chatbot that can help customers book or reschedule spa services. Instead of using a class-based approach , it uses a collection of separate functions with shared state dictionaries.

In [2]:
import numpy as np
import re
import spacy
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Load spaCy for entity extraction
nlp = spacy.load('en_core_web_sm')

# Global state dictionaries
conversation_states = {}
pending_bookings = {}
new_booking_dates = {}
new_booking_times = {}
booking_services = {}
booking_database = {}

# Define intents and example queries (training data)
intents_data = {
    "booking": [
        "I want to book a massage",
        "Can I schedule a facial for next week?",
        "How do I book a session?",
        "I'd like to make an appointment",
        "Book a Swedish massage for me",
        "I need to schedule a wellness package",
        "Can you help me book a service?",
        "I want to reserve a time slot",
        "How can I book a therapist?",
        "I need to make a booking"
    ],
    "rescheduling": [
        "Can I reschedule my booking?",
        "I need to change my appointment time",
        "How do I modify my booking date?",
        "I want to reschedule my massage",
        "Change my appointment to next week",
        "I can't make it on Friday, need to reschedule",
        "Is it possible to change my booking time?",
        "Reschedule my facial appointment",
        "Move my appointment to another day",
        "I need to reschedule my session"
    ],
    "cancellation": [
        "I want to cancel my booking",
        "How do I cancel my appointment?",
        "Can I get a refund if I cancel?",
        "Need to cancel my massage",
        "Cancel my facial appointment",
        "What's the cancellation policy?",
        "I need to cancel due to emergency",
        "How late can I cancel?",
        "Is there a fee for cancellation?",
        "Can you help me cancel my booking?"
    ],
    "pricing": [
        "How much does a massage cost?",
        "What are your prices?",
        "Can I get a price list?",
        "Do you have any discounts?",
        "How much is a facial?",
        "What's the cost of a wellness package?",
        "Are there any special offers?",
        "Pricing for couple massage",
        "Tell me about your rates",
        "What is your fee structure?"
    ],
    "information": [
        "What services do you offer?",
        "Tell me about your massages",
        "How long is a typical session?",
        "What should I wear?",
        "Do I need to bring anything?",
        "Are your therapists certified?",
        "Where are you located?",
        "What are your business hours?",
        "Do you have parking?",
        "Can I request a specific therapist?"
    ],
    "greeting": [
        "Hello",
        "Hi there",
        "Hey",
        "Good morning",
        "Good afternoon",
        "Good evening",
        "Hi, how are you?",
        "Hello, can you help me?",
        "Hey there",
        "Hi Blys"
    ],
    "thanks": [
        "Thank you",
        "Thanks for your help",
        "That's great, thanks",
        "Appreciate it",
        "Thanks a lot",
        "Thank you so much",
        "That's helpful, thanks",
        "Perfect, thank you",
        "Great, thanks",
        "Thanks for the information"
    ]
}

# Define response templates
response_templates = {
    "booking": [
        "I can help you book a session. What service are you interested in?",
        "Sure, I'd be happy to help you make a booking. What type of service would you like?",
        "I can assist with your booking. Which service would you prefer?"
    ],
    "rescheduling": [
        "Yes, you can reschedule your booking through the Blys app. Would you like me to assist you?",
        "I can help you reschedule your appointment. Would you like me to assist with that?",
        "Rescheduling is possible. Would you like me to help you with that?"
    ],
    "rescheduling_confirmation": [
        "Please provide the new date that you would like to reschedule your booking at.",
        "When would you like to reschedule your appointment for?",
        "Please let me know your preferred new date and time for your booking."
    ],
    "rescheduling_success": [
        "Sent reschedule information to pro, you will get notified once it's confirmed.",
        "Your reschedule request has been sent to the professional. You'll receive a confirmation notification soon.",
        "Rescheduling request submitted. The professional will confirm the new time and you'll be notified."
    ],
    "cancellation": [
        "I can help you cancel your booking. Please note that cancellations within 24 hours may incur a fee. Would you like to proceed?",
        "Sure, I can assist with cancellation. Please be aware of our 24-hour cancellation policy. Would you like to continue?",
        "I can help you cancel. Just a reminder that our cancellation policy requires 24 hours notice to avoid fees. Would you like to proceed with cancellation?"
    ],
    "pricing": [
        "Our pricing varies by service. Massages range from $80-$300, facials from $75-$150, and wellness packages from $200-$500. Which service are you interested in?",
        "I'd be happy to help with pricing information. We offer massages ($80-$300), facials ($75-$150), and packages ($200-$500). What service are you looking for?",
        "Our prices depend on the service and duration. Generally, massages are $80-$300, facials $75-$150, and packages $200-$500. Which specific service would you like to know about?"
    ],
    "information": [
        "We offer a range of wellness services including massages, facials, hair styling, and comprehensive wellness packages. Is there something specific you'd like to know about?",
        "Blys provides various services including massage therapy, facial treatments, hair styling, and wellness packages. What would you like more information about?",
        "Our services include various massage types, facial treatments, hair and makeup, and wellness packages. Can I provide specific information about any of these?"
    ],
    "greeting": [
        "Hello! Welcome to Blys. How can I assist you today?",
        "Hi there! Thank you for contacting Blys. How may I help you?",
        "Good day! This is Blys virtual assistant. What can I do for you today?"
    ],
    "thanks": [
        "You're welcome! Is there anything else I can help you with?",
        "My pleasure! If you have any other questions, feel free to ask.",
        "Happy to help! Is there anything else you need assistance with?"
    ],
    "default": [
        "I'm not sure I understand. Could you please rephrase that?",
        "I didn't quite catch that. Can you try asking in a different way?",
        "I'm still learning and didn't understand your request. Could you clarify?"
    ]
}

def prepare_training_data():

    texts = []
    labels = []

    for intent, examples in intents_data.items():
        for example in examples:
            texts.append(example)
            labels.append(intent)

    return train_test_split(texts, labels, test_size=0.2, random_state=42)

def train_intent_classifier():

    X_train, X_test, y_train, y_test = prepare_training_data()

    # Create an intent classification pipeline with TF-IDF and SVM
    intent_classifier = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
        ('clf', SVC(kernel='linear', probability=True))
    ])

    # Train the classifier
    intent_classifier.fit(X_train, y_train)

    return intent_classifier

def get_random_response(intent):

    templates = response_templates.get(intent, response_templates["default"])
    return np.random.choice(templates)

def predict_intent(intent_classifier, text):

    intent_probs = intent_classifier.predict_proba([text])[0]
    intent_idx = np.argmax(intent_probs)
    confidence = intent_probs[intent_idx]

    if confidence < 0.4:
        return "default"

    return intent_classifier.classes_[intent_idx]

def extract_date_time(text):

    doc = nlp(text)

    date_pattern = r'\b(\d{1,2})\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+(\d{4})\b'
    time_pattern = r'\b(\d{1,2})(:\d{2})?\s*(am|pm)\b'

    date_match = re.search(date_pattern, text, re.IGNORECASE)
    time_match = re.search(time_pattern, text, re.IGNORECASE)

    date_str = None
    time_str = None

    if date_match:
        date_str = date_match.group(0)

    if time_match:
        time_str = time_match.group(0)

    dates = []
    times = []

    for ent in doc.ents:
        if ent.label_ == "DATE":
            dates.append(ent.text)
        elif ent.label_ == "TIME":
            times.append(ent.text)

    if date_str is None and dates:
        date_str = dates[0]

    if time_str is None and times:
        time_str = times[0]

    return date_str, time_str

def update_booking(booking_id, new_date, new_time):

    if booking_id in booking_database:
        booking_database[booking_id]["date"] = new_date
        booking_database[booking_id]["time"] = new_time
        return True
    return False

def create_booking(service, date, time):

    booking_id = len(booking_database) + 1001
    booking_database[booking_id] = {
        "service": service,
        "date": date,
        "time": time,
        "status": "confirmed"
    }
    return booking_id

def process_message(intent_classifier, user_message, session_id="default"):

    if session_id not in conversation_states:
        conversation_states[session_id] = "start"
        pending_bookings[session_id] = None
        new_booking_dates[session_id] = None
        new_booking_times[session_id] = None
        booking_services[session_id] = None

    current_state = conversation_states[session_id]

    if current_state == "start":
        intent = predict_intent(intent_classifier, user_message)

        if intent == "rescheduling":
            conversation_states[session_id] = "confirm_reschedule"
            pending_bookings[session_id] = 1001
            return get_random_response("rescheduling")

        elif intent == "booking":
            conversation_states[session_id] = "ask_service"
            return get_random_response("booking")

        else:
            return get_random_response(intent)

    elif current_state == "confirm_reschedule":

        lower_msg = user_message.lower()
        if "yes" in lower_msg or "sure" in lower_msg or "okay" in lower_msg:
            conversation_states[session_id] = "ask_reschedule_date"
            return get_random_response("rescheduling_confirmation")
        else:
            conversation_states[session_id] = "start"
            return "I'll keep your booking as is. Is there anything else I can help with?"

    elif current_state == "ask_reschedule_date":

        date_str, time_str = extract_date_time(user_message)

        if date_str and time_str:

            new_booking_dates[session_id] = date_str
            new_booking_times[session_id] = time_str

            success = update_booking(pending_bookings[session_id], date_str, time_str)

            conversation_states[session_id] = "start"
            return get_random_response("rescheduling_success")

        elif date_str:

            new_booking_dates[session_id] = date_str
            conversation_states[session_id] = "ask_reschedule_time"
            return f"Got it, {date_str}. What time would you prefer?"

        else:

            return "I couldn't understand the date. Please provide a date in format like '30 Mar 2025'."

    elif current_state == "ask_reschedule_time":

        _, time_str = extract_date_time(user_message)

        if time_str:
            new_booking_times[session_id] = time_str

            success = update_booking(
                pending_bookings[session_id],
                new_booking_dates[session_id],
                time_str
            )

            conversation_states[session_id] = "start"
            return get_random_response("rescheduling_success")
        else:
            return "I couldn't understand the time. Please provide a time in format like '10 am' or '2:30 pm'."

    elif current_state == "ask_service":

        booking_services[session_id] = user_message
        conversation_states[session_id] = "ask_booking_date"
        return f"Great choice! When would you like to book the {user_message}?"

    elif current_state == "ask_booking_date":

        date_str, time_str = extract_date_time(user_message)

        if date_str and time_str:

            booking_id = create_booking(
                booking_services[session_id],
                date_str,
                time_str
            )

            conversation_states[session_id] = "start"
            return f"Your booking for {booking_services[session_id]} on {date_str} at {time_str} is confirmed! Your booking reference is #{booking_id}."

        elif date_str:

            new_booking_dates[session_id] = date_str
            conversation_states[session_id] = "ask_booking_time"
            return f"Got it, {date_str}. What time would you prefer?"

        else:

            return "I couldn't understand the date. Please provide a date in format like '30 Mar 2025'."

    elif current_state == "ask_booking_time":

        _, time_str = extract_date_time(user_message)

        if time_str:

            booking_id = create_booking(
                booking_services[session_id],
                new_booking_dates[session_id],
                time_str
            )

            conversation_states[session_id] = "start"
            return f"Your booking for {booking_services[session_id]} on {new_booking_dates[session_id]} at {time_str} is confirmed! Your booking reference is #{booking_id}."
        else:
            return "Please provide a time in format like '10 am' or '2:30 pm'."


    return "I'm not sure I understand. Can you please try again?"

def save_model(intent_classifier):

    with open('chatbot_model.pkl', 'wb') as f:
        pickle.dump({
            'intent_classifier': intent_classifier,
            'response_templates': response_templates
        }, f)
    print("Model saved as chatbot_model.pkl")

def load_model():

    with open('chatbot_model.pkl', 'rb') as f:
        model_data = pickle.load(f)
    return model_data['intent_classifier'], model_data['response_templates']

def reset_conversation(session_id="default"):
    """Reset a conversation to its initial state"""
    conversation_states[session_id] = "start"
    pending_bookings[session_id] = None
    new_booking_dates[session_id] = None
    new_booking_times[session_id] = None
    booking_services[session_id] = None

def get_booking_details(booking_id):

    if booking_id in booking_database:
        return booking_database[booking_id]
    return None

def main():

    print("Training intent classifier")
    intent_classifier = train_intent_classifier()

    print("Saving model")
    save_model(intent_classifier)

    print("Testing Conversation")
    test_conversation = [
        "Hi there",
        "Can I reschedule my booking?",
        "Yes",
        "30 Mar 2025 10 am",
        "Thank you"
    ]

    session_id = "test_session"
    for message in test_conversation:
        print(f"User: {message}")
        response = process_message(intent_classifier, message, session_id)
        print(f"Bot: {response}")
        print()

if __name__ == "__main__":
    main()

Training intent classifier
Saving model
Model saved as chatbot_model.pkl
Testing Conversation
User: Hi there
Bot: Hi there! Thank you for contacting Blys. How may I help you?

User: Can I reschedule my booking?
Bot: Yes, you can reschedule your booking through the Blys app. Would you like me to assist you?

User: Yes
Bot: Please let me know your preferred new date and time for your booking.

User: 30 Mar 2025 10 am
Bot: Sent reschedule information to pro, you will get notified once it's confirmed.

User: Thank you
Bot: You're welcome! Is there anything else I can help you with?

